# **III. Proteins**
## The goal here is to generate the Df for the final analysis 
### I. DF with the proteins from minibatch and ppt with the relevant info
### II. DF 
***

In [ ]:
import pandas as pd 
import os 
import json
from tqdm import tqdm

path_labels = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_labeling/phageboost"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"


final_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.final.csv", sep="\t", header = 0)
df_labels = pd.read_csv(f"{path_labels}/info/prophage_data.clusters_80.phageboost_70.2504.tsv", sep = "\t" , skiprows=1)
df_labels.columns = ["Prophage_name","KL_type","Infected_ancestor","n_clades","siblings","n_ancestors","n_KL_swaps","old_KL_types","all_old_KL_types"]

final_df = final_df.drop_duplicates(subset = ["protein_name","index_seq"] , keep = "first")

# Write the ppt part :
with open(f"{path_db}/DF_Dpo.final.1005.tsv", "w") as outfile :
    outfile.write(f"Phage\tKL_type_LCA\tInfected_ancestor\tProtein_name\tDataset\tindex\tseq\n")
    for _,row in tqdm(final_df.iterrows()) :
        phage = "__".join(row["protein_name"].split("__")[0:-1])
        KL_type = df_labels[df_labels["Prophage_name"] == f"{phage}.fasta"]["KL_type"].values[0]
        infected_ancestor = df_labels[df_labels["Prophage_name"] == f"{phage}.fasta"]["Infected_ancestor"].values[0]
        db_id = f"ppt__{row['index_seq']}"
        outfile.write(f"{phage}\t{KL_type}\t{infected_ancestor}\t{row['protein_name']}\tppt\t{db_id}\t{row['sequence']}\n")
    
    
"""# Check that the prot name is not present in the ppt set :
ppt_df = pd.read_csv(f"{path_db}/DF_Dpo.final.1005.tsv", sep = "\t", header = 0)
minibatch_prot_seq = json.load(open(f"{path_db}/new_mini_batch.prophage_raw.json"))

for prot_name in minibatch_prot_seq :
    if prot_name in ppt_df["Protein_name"].to_list() :
        print("TF" , prot_name)
# No hits ==> Good        
"""

minibatch_prot_seq = json.load(open(f"{path_db}/new_mini_batch.prophage_raw.json"))
minibatch_index = pd.read_csv(f"{path_db}/minibatch_index.csv", sep = "\t" , names = ["index_seq", "sequence"])
# Write the minibatch part :
with open(f"{path_db}/DF_Dpo.final.1005.tsv", "a+") as outfile :
    for sequence,prot_names in tqdm(minibatch_prot_seq.items()) :
        for prot_name in prot_names : 
            phage = "__".join(prot_name.split("__")[0:-1])
            KL_type = df_labels[df_labels["Prophage_name"] == f"{phage}.fasta"]["KL_type"].values[0]
            infected_ancestor = df_labels[df_labels["Prophage_name"] == f"{phage}.fasta"]["Infected_ancestor"].values[0]
            try : 
                minibatch_id = minibatch_index[minibatch_index["sequence"] == sequence]["index_seq"].values[0]
                db_id = f"minibatch__{minibatch_id}"
                outfile.write(f"{phage}\t{KL_type}\t{infected_ancestor}\t{prot_name}\tminibatch\t{db_id}\t{sequence}\n")
            except :
                break
    

In [ ]:
#